In [2]:
from selenium import webdriver
from bs4 import BeautifulSoup
from selenium.webdriver.common.by import By 
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC 
import openpyxl
import pandas as pd 
from datetime import datetime
import re
import time 

driver = webdriver.Chrome('C:/Users/concat/Desktop/LinaCrawling/CONCAT/chromedriver_win32/chromedriver.exe')
url = 'https://online.abllife.co.kr/insurance-product/health-coverage/dental.abl?utm_campaign=dental&utm_source=insmarket-p&utm_medium=referral&utm_content=ebiz&utm_term=main_dental'

driver.get(url)

In [8]:
# 성별
#driver.find_element_by_class_name('man').click()
driver.find_element_by_class_name('woman').click()

In [5]:
# 생년월일
# yyyymmdd
# 만 19세 ~ 만 70세 까지
# 60세 이상은 10년 만기만 가능 

driver.find_element_by_id('brthDay').send_keys('19961129')
#driver.find_element_by_id('brthDay').clear()



In [10]:
# 보험료 확인 버튼
driver.find_element_by_id('calcStrBtn').click()

In [ ]:
# loading bar
wait = WebDriverWait(driver,20)

wait.until(EC.invisibility_of_element((By.id,'globalLoadingDiv')))


In [64]:
page = driver.page_source
soup = BeautifulSoup(page,'html.parser')

# 보험료 및 보장내역이 나와있는 Box
result = soup.find(id = 'direcEntplDiv')

# 보험 기간 등의 정보가 있는 Box
planBox = result.find(id = 'direcContDiv')

# 보험 가입금액
joinPrice = planBox.find(id = "mnContEntAmt00Div")

# 보험 기간
period = planBox.find(id = "mnInsrPrdYys00Div")

# 납입 기간
payment = planBox.find(id = "mnInsrPadPrdYys00Div")




In [65]:
period.find()

<li id="mnInsrPrdYys00Div">
<strong><label for="mnInsrPrdYys00">보험기간</label></strong>
<span>
<select class="sel_m" id="mnInsrPrdYys00"><option value="0">10년</option><option value="1">20년</option></select>
</span>
<span class="mnInsrPrdYys_st subtxt" style="">최대 80세까지 자동갱신 및 보장</span>
</li>

In [55]:
# 보장 내역
guaranteeList = result.find(id = "entplFinDiv").find(class_='box_area')

#보험료
premium = result.find(id = "entplFinDiv").find(class_='box2_area')


In [21]:
# planBox Crawling
title = joinPrice.find('strong').get_text()

In [22]:
title

'주계약 보험가입금액'

In [23]:
joinPrice

<li id="mnContEntAmt00Div">
<strong><label for="mnContEntAmt00">주계약 보험가입금액</label></strong>
<span>
<select class="sel_m" disabled="disabled" id="mnContEntAmt00"><option value="5000000">500만원</option></select>
</span>
<span class="mnContEntAmt_st subtxt" style="display:none"></span>
</li>

In [26]:
content = joinPrice.find(class_='sel_m').get_text()

In [27]:
content

'500만원'

In [29]:
# 보험기간
period = driver.find_element_by_id("mnInsrPrdYys00Div")

In [30]:
period_options = period.find_elements_by_tag_name('option')

In [33]:
for option in period_options :
    print(option.text)

10년
20년


In [35]:
# 납입 기간 == 보험 기간 

In [41]:
# 보장 내역
notwants = guaranteeList.find_all(class_='stnrdPrdtList')
for notwant in notwants : 
    notwant.extract()
tr_list = guaranteeList.find_all(class_= 'insurance_info_dental')


In [46]:
tr_list[0]

<div class="insurance_info_dental" id="benfCnt1"><span> </span></div>

In [48]:
content = dict()

for tr in tr_list : 
    title = tr.find('strong')
    price = tr.find('span')
    
    if title == None :
        continue
    
    title = title.get_text()
    price = price.get_text()
    content[title] = price

In [49]:
content

{'임플란트': '50만원',
 '틀니치료': '50만원',
 '브릿지': '25만원',
 '인레이/온레이': '10만원',
 '복합레진': '7.5만원',
 '아말감/GI': '1.5만원',
 '치조골이식술': '25만원',
 '크라운치료': '15만원'}

In [62]:
# 보험료
premium_price = premium.find(id = "vwMnContPrm00").get_text()

In [63]:
premium_price

'11,600'

In [6]:
def getContent(page,content) : 

    soup = BeautifulSoup(page,'html.parser')

    result = soup.find(id = 'direcEntplDiv')

    # 보장 내역 and 보험료
    guaranteeList = result.find(id = "entplFinDiv").find(class_='box_area')
    premium = result.find(id = "entplFinDiv").find(class_='box2_area')

    # 원하지 않는 요소들을 제외시켜줍니다.
    not_wants = guaranteeList.find_all(class_='stnrdPrdtList')
    for nots in not_wants : 
        nots.extract()
    
    # 보장 내역을 뽑아와 이름과 내역을 매칭시켜줍니다. 
    gt_list = guaranteeList.find_all(class_='insurance_info_dental')

    for gt in gt_list : 
        title = gt.find('strong')
        price = gt.find('span')

        if title == None :
            continue
        
        title = title.get_text()
        price = price.get_text()
        content[title] = price
    
    premium_price = premium.find(id = "vwMnContPrm00").get_text()

    content['보험료'] = premium_price

    return content

In [43]:
driver = webdriver.Chrome('C:/Users/concat/Desktop/LinaCrawling/CONCAT/chromedriver_win32/chromedriver.exe')
url = 'https://online.abllife.co.kr/insurance-product/health-coverage/dental.abl?utm_campaign=dental&utm_source=insmarket-p&utm_medium=referral&utm_content=ebiz&utm_term=main_dental'

driver.get(url)

driver.set_window_size(1920,1028)

# 가입 연령은 만 19세부터 만 70세까지입니다.
young = 20
old   = 70 

year = datetime.now().year

young = int(str(year - young) + '0728')
old   = int(str(year - old)   + '0728')

wait = WebDriverWait(driver,20)

contents = list()

for age in range(old,young,50000) : 
    driver.find_element_by_id('brthDay').clear()
    driver.find_element_by_id('brthDay').send_keys(age)

    # 성별 선택
    for i in range(2) : 
        if i == 0 : 
            driver.find_element_by_class_name('man').click()
            sex = '남자'
        else : 
            driver.find_element_by_class_name('woman').click()
            sex = '여자'

        # 보험료 계산하기
        driver.find_element_by_id('calcStrBtn').click()

        wait.until(EC.invisibility_of_element((By.ID,'globalLoadingDiv')))
        time.sleep(1)
        
        # 보험 기간 선택하기
        period = driver.find_element_by_id("mnInsrPrdYys00Div")
        period_options = period.find_elements_by_tag_name('option')
        
        try : 
            driver.find_element_by_id("direcEntplTab").click()
        except :
            pass 

        for option in period_options :
            option.click()
            time.sleep(2)
            
            content = dict()
            content['생년월일'] = age
            content['성별'] = sex 
            content['주계약 보험금액'] = '500만원'
            content['보험 기간'] = option.text
            content['납입 기간'] = option.text

            page = driver.page_source

            content = getContent(page,content)
            contents.append(content)

df = pd.DataFrame(contents, columns = contents[0].keys())
df.to_excel("ABL.xlsx")

In [14]:
20190728 - 19590728

600000

In [30]:
contents

[{'생년월일': 19490728,
  '성별': '남자',
  '주계약 보험금액': '500만원',
  '보험 기간': '10년',
  '납입 기간': '10년',
  '임플란트': '50만원',
  '틀니치료': '50만원',
  '브릿지': '25만원',
  '인레이/온레이': '10만원',
  '복합레진': '7.5만원',
  '아말감/GI': '1.5만원',
  '치조골이식술': '25만원',
  '크라운치료': '15만원',
  '보험료': '46,550'},
 {'생년월일': 19490728,
  '성별': '여자',
  '주계약 보험금액': '500만원',
  '보험 기간': '10년',
  '납입 기간': '10년',
  '임플란트': '50만원',
  '틀니치료': '50만원',
  '브릿지': '25만원',
  '인레이/온레이': '10만원',
  '복합레진': '7.5만원',
  '아말감/GI': '1.5만원',
  '치조골이식술': '25만원',
  '크라운치료': '15만원',
  '보험료': '34,550'},
 {'생년월일': 19540728,
  '성별': '남자',
  '주계약 보험금액': '500만원',
  '보험 기간': '10년',
  '납입 기간': '10년',
  '임플란트': '50만원',
  '틀니치료': '50만원',
  '브릿지': '25만원',
  '인레이/온레이': '10만원',
  '복합레진': '7.5만원',
  '아말감/GI': '1.5만원',
  '치조골이식술': '25만원',
  '크라운치료': '15만원',
  '보험료': '46,100'},
 {'생년월일': 19540728,
  '성별': '여자',
  '주계약 보험금액': '500만원',
  '보험 기간': '10년',
  '납입 기간': '10년',
  '임플란트': '50만원',
  '틀니치료': '50만원',
  '브릿지': '25만원',
  '인레이/온레이': '10만원',
  '복합레진': '7.5만원',
  '아말감/GI': '1.5

In [26]:
driver.find_element_by_id("mnContEntAmt00Div").find_element_by_class_name("sel_m").text

'500만원'

In [38]:
tests = list()
test = dict()
# 보험 기간 선택하기
period = driver.find_element_by_id("mnInsrPrdYys00Div")
period_options = period.find_elements_by_tag_name('option')

try : 
    driver.find_element_by_id("direcEntplTab").click()
except :
    pass 

for option in period_options :
    option.click()
    time.sleep(2)
    print(option.text)
    test['보험 기간'] = option.text
    test['납입 기간'] = option.text
    
    tests.append(test)

10년
20년


In [39]:
test

{'보험 기간': '20년', '납입 기간': '20년'}

In [40]:
tests

[{'보험 기간': '20년', '납입 기간': '20년'}, {'보험 기간': '20년', '납입 기간': '20년'}]

In [41]:
test['보험 기간'] = "ㅋㅋ"

In [42]:
tests

[{'보험 기간': 'ㅋㅋ', '납입 기간': '20년'}, {'보험 기간': 'ㅋㅋ', '납입 기간': '20년'}]